##Installing DeepChem

In [2]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3501  100  3501    0     0  92131      0 --:--:-- --:--:-- --:--:-- 92131


python version: 3.7.10
remove current miniconda
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [3]:
!pip install --pre deepchem
!pip install dgl
!pip install dgllife

     |████████████████████████████████| 552kB 8.4MB/s 
     |████████████████████████████████| 4.3MB 8.0MB/s 
     |████████████████████████████████| 133kB 6.9MB/s 
  Created wheel for dgllife: filename=dgllife-0.2.6-cp37-none-any.whl size=205837 sha256=f0d52e48f594ab9438752558be7cb065b1ce28e0889464262b3242d91b5e05ee
  Stored in directory: /root/.cache/pip/wheels/cf/29/39/54bbf37c8999f605d347805d4dec838e9ca8a894c90b119600
Successfully built dgllife


We can now import the `deepchem` package to play with.

In [10]:
import deepchem as dc
dc.__version__

'2.6.0.dev'

## Using GraphConv featuriser

Implementing and recording the baseline for Tox21 dataset.

In [11]:
featurizer = dc.feat.MolGraphConvFeaturizer()
tasks, datasets, transformers = dc.molnet.load_lipo(featurizer=featurizer)
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (3360,), y.shape: (3360, 1), w.shape: (3360, 1), task_names: ['exp']>


ECFP featurizer is used. Extended Connectivity Fingerprints  is a **fingerprinting** method. They are also sometimes called "circular fingerprints". The ECFP algorithm begins by classifying atoms based only on their direct properties and bonds. Each unique pattern is a feature. For example, "carbon atom bonded to two hydrogens and two heavy atoms" would be a feature, and a particular element of the fingerprint is set to 1 for any molecule that contains that feature. It then iteratively identifies new features by looking at larger circular neighborhoods. One specific feature bonded to two other specific features becomes a higher level feature, and the corresponding element is set for any molecule that contains it. This continues for a fixed number of iterations, most often two.

In [12]:
tasks

['exp']

Above are the tasks in the Tox21 dataset - there are 12 tasks, each corresponding to different biotoxicity targets, such as cell receptors and stress response pathways.

In [13]:
datasets

(<DiskDataset X.shape: (3360,), y.shape: (3360, 1), w.shape: (3360, 1), task_names: ['exp']>,
 <DiskDataset X.shape: (420,), y.shape: (420, 1), w.shape: (420, 1), ids: ['Nc1nccc(\\C=C\\c2ccc(Cl)cc2)n1'
  'CC(=O)Nc1ccc2ccn(c3cc(NCCCN4CCCC4)n5ncc(C#N)c5n3)c2c1'
  'OC(CN1CCOCC1)c2ccccc2' ...
  'COc1cc(CCc2cc(Nc3ccnc(NCc4ccccn4)n3)n[nH]2)cc(OC)c1'
  'CCN(CC)C(=O)c1ccc(cc1)C(=C2CCN(Cc3ccc(F)cc3)CC2)c4cccnc4'
  'CC(C)(C(=O)Nc1oc(nn1)C(=O)Nc2ccc(cc2)N3CCOCC3)c4ccc(Cl)cc4'], task_names: ['exp']>,
 <DiskDataset X.shape: (420,), y.shape: (420, 1), w.shape: (420, 1), ids: ['O[C@@H](CNCCCOCCNCCc1cccc(Cl)c1)c2ccc(O)c3NC(=O)Sc23'
  'NC1=NN(CC1)c2cccc(c2)C(F)(F)F'
  'COc1cc(ccc1Cn2ccc3ccc(cc23)C(=O)NCC4CCCC4)C(=O)NS(=O)(=O)c5ccccc5' ...
  'CCCSc1ncccc1C(=O)N2CCCC2c3ccncc3' 'Oc1ncnc2scc(c3ccsc3)c12'
  'OC1(CN2CCC1CC2)C#Cc3ccc(cc3)c4ccccc4'], task_names: ['exp']>)

There are three dataset objects - train split, val split and test split. Each split consists of X and y - X is the features and y is the output label. Froe example the train split has X.shape (513, 1024)
and y.shape (513, 1). This implies that there are 513 samples in the train split - and each sample is represented by an ECFP vector of size 1024. 

##Training the GCNNN

In [20]:
from deepchem.models import GCNModel
model = GCNModel(mode='regression', n_tasks=len(tasks), batch_size=32, learning_rate=0.001)
model.fit(train_dataset, nb_epoch=50)

0.15392576217651366

`MultitaskClassifier` is a simple stack of fully connected layers. A single hidden layer of width 1000 is used. Each input will have 1024 features, and it should produce predictions for 12 different tasks.

Note that the above network is performing multitask learning - a single network is used for all 12 tasks. This is because inter task correlations exist in the data, and to take if advantage of this single neural network is used for multiple tasks.

In [21]:
import numpy as np

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(test_dataset, [metric], transformers))

training set score: {'pearson_r2_score': 0.8298407416552742}
test set score: {'pearson_r2_score': 0.515059956498543}


The training set score is much higher than test set score. This indicates overfitting - and is why metrics on the validation set need to be measured in otder to tune parameters and detect overfitting.